In [1]:
!pip install gym==0.21 > /dev/null
!pip install stable-baselines3[extra] box2d > /dev/null

In [2]:
!pip install pyvirtualdisplay > /dev/null
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null

In [3]:
import random 
import gym
from distutils.dir_util import copy_tree
from stable_baselines3 import DQN
from collections import OrderedDict

In [4]:
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
import glob
import io
import base64
from gym.wrappers import Monitor
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [5]:
env = gym.make("LunarLander-v2")
# env.seed()


In [6]:
model = DQN(policy='MlpPolicy', env=env, batch_size=128, buffer_size=50000, exploration_final_eps=0.1, exploration_fraction=0.12, gamma=0.99, gradient_steps=-1, learning_rate=0.00063,
            learning_starts=0, policy_kwargs=dict(net_arch=[256, 256]), target_update_interval=250, train_freq=4)
model.learn(total_timesteps=100000)
model.save("lunarlander_dqn")
# model.load("lunarlander_hf_dqn")

In [7]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video(folder):
  mp4list = glob.glob(f'{folder}/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env, folder):
  env = Monitor(env, folder, force=True)
  return env

In [8]:
def simulate(env, model, verbose=False):
  observation = env.reset()
  done = False
  i = 0
  while not done:
    i += 1
    action = model.predict(observation)[0]
    observation, reward, done, info = env.step(action)
    if verbose and (done or i % 500):
      display_results(observation, reward, done, info)
  return i, reward >= 100
      

def display_results(observation, reward, done, info):
  print(f"Observation: {observation}")
  print(f"Reward: {reward}")
  print(f"Done: {done}")
  print(f"Info: {info}")
  print("==================================================")



---



In [9]:
# video_folder = './video'
# env = wrap_env(env, video_folder)
# steps, win = simulate(env, model)
# env.close()
# show_video(video_folder)

In [10]:
best_steps = 1e9
for i in tqdm(range(50)):
  env = wrap_env(env, './video')
  steps, win = simulate(env, model)
  env.close()
  if steps < best_steps and win:
    best_steps = steps
    copy_tree('./video', f'./video_{best_steps}') 
print(best_steps)


100%|██████████| 50/50 [02:53<00:00,  3.47s/it]

194




---



In [11]:
show_video(f'video_{best_steps}')

# number = 62
# show_video(f'video_{number}')